In [4]:
import sklearn
import pandas as pd
import numpy as np

### Datensatz laden

In [10]:
# Datensatz laden

daten = pd.read_json("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_HandlungDetail.json")
daten = daten.drop(axis=1, labels=["corpus_id"])
print(daten)

                label                                              tweet
0     [KeineHandlung]  @user @user @user Weitaus schlimmer. Heute ist...
1     [KeineHandlung]   Das Deutsche Kaiserreich soll wieder auferstehen
2     [KeineHandlung]                   Die BRD ist eine einzige Schande
3     [KeineHandlung]  @user @user Die Grünen....besser kann man das ...
4     [KeineHandlung]  @user @user Scheiss deutsche Politiker! Mehr g...
...               ...                                                ...
3045  [KeineHandlung]  Unbequeme Wahrheit:   Sexuelle Belästigung ist...
3046  [KeineHandlung]  Vor was habt ihr Angst Liebe Bürger !!!???? St...
3047  [KeineHandlung]  @user Schön den Dummkopf #Oppermann von der Vo...
3048  [KeineHandlung]  @user   Hoffentlich sind die Nationalisten dan...
3049  [KeineHandlung]  @user Klar, was sollen Sie denn auch anderes s...

[3050 rows x 2 columns]


### Annotationslabels encodieren

In [40]:
# Annotationen konvertieren

def transform_label(label):
    ja = ["NEG", "VVH", "Gruppe", "Handlung", "Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
        "Politische Einstellung", "Geschlecht", "Anderes Merkmal", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"]
    
    if label in ja: return 1
    else: return 0

def encode(labels):
    """TODO
    # ausgehend nur von daten["label"]:
    # 1. eine Liste der Labelnamen ausgeben (also herausfinden, um welches Klassifikationsproblem es genau geht)
    # 2. die Labels in eine Liste der transformierten Labels verwandeln

    """

    # 1. Klassifizierungsproblem, also das Set der vorhandenen Labels, ermitteln
    labels_vvh = ["KEINE", "VVH"]
    labels_gruppe = ["KeineGruppe", "Gruppe"]
    labels_handlung = ["KeineHandlung", "Handlung"]
    labels_gruppe_det = ["KeineGruppe", "Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
        "Politische Einstellung", "Geschlecht", "Anderes Merkmal"]
    labels_handlung_det = ["KeineHandlung", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"]
    label_namen = [labels_vvh, labels_gruppe, labels_handlung, labels_gruppe_det, labels_handlung_det]

    # Fall 1: Strings (binäre Klassen)
    namen = []
    if type(labels[0]) == str:
        labels_flat = set(labels)
    # Fall 2: Listen (mehrere Klassen)
    else:
        # Summe über die Sets aller Listen
        labels_flat = set([label for entry in labels for label in entry])
    for i in label_namen:
        if set(i) == labels_flat: namen = i

    # 2. Labels transformieren

    # Fall 1: nur 1 Klasse
    if len(namen) == 2:
        return ([list(map(transform_label, list(labels)))], namen)

    # Fall 2: mehrere Klassen
    else:
        labels_bin = []
        # neue Liste von Labels erstellen, aus der alle anderen Klassen entfernt sind
        labels_sep = []
        for nom in namen[1:]:
            labels_neu = []
            for entry in labels:
                if nom in entry:
                    labels_neu.append(nom)
                else: labels_neu.append(namen[0])
            labels_sep.append(labels_neu)
        # für jede Klasse die Labels transformieren
        for klasse in labels_sep:
            labels_bin.append(list(map(transform_label, klasse)))
        return (labels_bin, namen)


# TODO
def decode(namen, labels):
    """ # ausgehend von den vorhergesagten Labels und der Liste der Labelname
    # 1. die Labels wieder von 0,1 in ihre Strings verwandeln

    """
    return True

labels_encoded, label_namen = encode(daten["label"])
print(label_namen)

['KeineHandlung', 'Aufstachelung zu Hass', 'Aufforderung zu Gewalt- oder Willkürmaßnahmen', 'Angriff der Menschenwürde']


### Tokenizer für die Klassifikation mit sklearn laden

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('deepset/gelectra-large', truncation=True, padding=False)

def bert_tokenize(inputs):
    """Einen String mit BERT tokenisieren
    Output: Liste von Tokens"""
    token_ids = tokenizer(inputs)
    tokens = tokenizer.convert_ids_to_tokens(token_ids["input_ids"])
    return tokens   

Downloading: 100%|██████████| 443/443 [00:00<00:00, 139kB/s]
Downloading: 100%|██████████| 240k/240k [00:01<00:00, 195kB/s]  
Downloading: 100%|██████████| 83.0/83.0 [00:00<00:00, 24.5kB/s]


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve

def pipeline(daten, ziele, label_set):
    """
    Train/Test-Split, Preprocessing und Merkmalsauswahl, Training und Evaluation
    Input: vollständiger Datensatz (daten) + Annotation (ziele)
    Output: Ergebnisse der Evaluation
    """
    # 1. Stratifizierter Train/Test-Split
    X_train, X_test, y_train, y_test = train_test_split(daten["tweet"], ziele, test_size=0.33, random_state=36, stratify=ziele)

    # 2. Preprocessing und Merkmalsauswahl
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), lowercase=True, tokenizer=bert_tokenize)
    X_train_feat = vectorizer.fit_transform(X_train)
    X_test_feat = vectorizer.transform(X_test)

    # 3. Normalisierung
    max_abs_scaler = preprocessing.MaxAbsScaler()
    X_train_maxabs = max_abs_scaler.fit_transform(X_train_feat)
    X_test_maxabs = max_abs_scaler.transform(X_test_feat)

    # 4. Training
    model = LogisticRegression()
    model.fit(X_train_maxabs, y_train)    

    # 5. Evaluation
    evaluation = dict() # Precision, Recall, Accuracy, F1, MCC, Confusion Matrix

    predicted = model.predict(X_test_maxabs)
    # metrics.classification_report(y_test, predicted, label_set)
    evaluation["confusion"] = metrics.confusion_matrix(y_test, predicted)

    return evaluation

### Pipeline für eine oder mehrere Klassen laufen lassen

In [41]:
ergebnisse = []


for i in range(len(label_namen)-1):
    ergebnisse.append(pipeline(daten, labels_encoded[i], [label_namen[0], label_namen[i+1]]))

# globale Evaluation
print(ergebnisse)

3
{'confusion': array([[1004,    0],
       [   3,    0]], dtype=int64)}
{'confusion': array([[990,   0],
       [ 17,   0]], dtype=int64)}
{'confusion': array([[995,   0],
       [ 12,   0]], dtype=int64)}
[{'confusion': array([[1004,    0],
       [   3,    0]], dtype=int64)}, {'confusion': array([[990,   0],
       [ 17,   0]], dtype=int64)}, {'confusion': array([[995,   0],
       [ 12,   0]], dtype=int64)}]
